In [1]:
from py import sqlite_utils as utils

In [2]:
conn = utils.connect_sqlite()

### Prediction metrics

In [3]:
utils.execute_sql_query("""
WITH max_versions AS (
    SELECT model_name, MAX(model_ver) AS max_model_ver
    FROM model_predictions
    GROUP BY model_name
), all_preds AS (
    SELECT mp.model_name, mp.hour, mp.pred_value
    FROM model_predictions AS mp 
    JOIN max_versions AS mv
    ON mp.model_name = mv.model_name
    WHERE mp.model_ver = mv.max_model_ver
), train_thre AS (
    SELECT MAX(hour) AS thre
    FROM power_features_train
), resids AS (
    SELECT
        ap.model_name,
        ap.hour,
        CASE WHEN ap.hour <= (SELECT thre FROM train_thre)
        THEN 'train' ELSE 'test' END AS subset,
        pf.y_kw_mean - ap.pred_value AS resid
    FROM all_preds AS ap
    JOIN power_features AS pf
    ON ap.hour = pf.hour
)
SELECT
    model_name,
    subset,
    COUNT(resid) AS n,
    SQRT(AVG(POWER(resid, 2))) AS rmse,
    AVG(ABS(resid)) AS mae,
    AVG(resid) AS bias
FROM resids
WHERE resid IS NOT NULL
GROUP BY model_name, subset
""").set_index(['model_name', 'subset']).unstack(level=1).sort_values(('mae', 'test'), axis=0)

n             rmse                 mae            \
subset                 test  train      test     train      test     train   
model_name                                                                   
catboost_mae           6679  27489  0.449683  0.502378  0.294591  0.301932   
baseline_catboost_mae  6679  27489  0.447051  0.446459  0.297019  0.253047   
catboost               6679  27489  0.450242  0.495193  0.311755  0.334068   
baseline_catboost      6679  27489  0.466513  0.378780  0.347381  0.258316   
baseline_lag1          6676  27483  0.600449  0.697311  0.394705  0.451494   
baseline_ffill_lag1    6679  27488  0.600844  0.697387  0.394940  0.451581   

                           bias            
subset                     test     train  
model_name                                 
catboost_mae           0.022408  0.075995  
baseline_catboost_mae  0.006314  0.069639  
catboost              -0.016136  0.000330  
baseline_catboost     -0.129531  0.000047  
baseline_lag1         -0.000274 -0.000247  
baseline_ffill_lag1    0.000086 -0.000141